<a href="https://colab.research.google.com/github/oguchi-ebube/Sugar-Instance-Packager/blob/master/Multinomial_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install chart_studio

In [0]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import scipy.stats
from scipy.stats import ttest_ind,mannwhitneyu,wilcoxon
import seaborn as sns
sns.set(style="ticks",color_codes=True)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
import gzip

In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/load_data.py" load_data.py
import load_data
!cp "/content/drive/My Drive/Colab Notebooks/data_helper_first.py" data_helper_first.py
import data_helper_first

In [0]:
def parse_gz(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def convert_to_DF(path):
    i = 0
    df = {}
    for d in parse_gz(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [8]:
# #insert path
path_df1 = '/content/drive/My Drive/Colab Notebooks/Products_review.csv'
path_df2 = '/content/drive/My Drive/Colab Notebooks/Products_reviews.csv'
path_df3 = '/content/drive/My Drive/Colab Notebooks/customer_review.csv'
path_df4 = '/content/drive/My Drive/Colab Notebooks/reviews_Sports_and_Outdoors_5.json.gz'


# Read the data in the CSV file using pandas
df1 = pd.read_csv(path_df1)
#print(df3)
df2 = pd.read_csv(path_df2)

df3 = pd.read_csv(path_df3, encoding='unicode_escape')
#print(df3)

!python load_data.py

df4 = convert_to_DF(path_df4)
# print(df4.columns)
df4 = df4.rename(index=str, columns={"reviewText": "review"})
df4 = df4.rename(index=str, columns={"overall": "rating"})
#print(df4.columns)

Data Succesfully Retrieved!


In [0]:
#concat the dataframes
df = pd.concat([df1, df2,df3,df4])
#df.drop(columns=['asin', 'asins', 'brand', 'categories', 'date', 'dateAdded','dateUpdated', 'helpful', 'imageURLs', 'keys', 'manufacturer','manufacturerNumber', 'name', 'primaryCategories',  'reviewTime', 'reviewerID', 'reviewerName', 'reviews.date','reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend','reviews.numHelpful', 'reviews.sourceURLs', 'reviews.title','reviews.username', 'sourceURLs', 'summary', 'title', 'unixReviewTime'])

# print(df[:5])
# print(df.columns)

df["label"] = np.where(df.rating >=3, 1, 0)



#print(df['label'])


#Number of the smaller class (negative reviews)
num_to_sample = len(df[df.label == 0])



df_neg = df[df["label"] == 0].sample(n=num_to_sample)


df_pos = df[df["label"] == 1].sample(n=num_to_sample)

In [0]:
df = pd.concat([df_neg, df_pos])


# Get the text from the dataframe
text = df["review"].values
#print(text)

# #clean strings
text= [data_helper_first.clean_str(str(sent)) for sent in text]



# Create labels from the dataframe
labels = df["label"].values


df = pd.concat([pd.DataFrame(text),pd.DataFrame(labels)],axis=1)
#print(len(df)) 144634
#print(df[3000:])

In [11]:
X = text
y = labels

print(df[600:])

                                                       0  0
600    the 10 and 20 codes have been unavailable for ...  0
601    i recently had surgery and am at the point tha...  0
602    i usually buy the 10 code for everything runni...  0
603    the power switch is good , the size is good , ...  0
604    this is the second one i own it is a bargain b...  0
...                                                  ... ..
91127  this is my goto knife that i always carry with...  1
91128  amazon wants to know what i think about this i...  1
91129  i originally became acquainted with concept2 a...  1
91130  we were looking for some back yard fun that wo...  1
91131  bought this watch 2 month ago did n't notice a...  1

[90532 rows x 2 columns]


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [0]:
#change text lower cases and removal of white spaces
lower_text = []
for i in range(0,len(X)):
  s = str(X[i])
  s1 = s.strip()
  lower_text.append(s1.lower())
  #print(“After converting text to lower case\n\n”,lower_text)

In [0]:
import re

In [0]:
#Remove punctuation
punc_text = []
for i in range(0,len(lower_text)):
  s2 = (lower_text[i])
  s3 = re.sub(r'[^\w\s2]',"",s2)
  punc_text.append(s3)
  # print("After removed punctuation\n\n",punc_text[:50])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
#Word vectorization
#Initialize the TF-IDF vectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_df = 0.7,norm='l2', encoding='latin-1', ngram_range=(1, 2),stop_words='english')

In [18]:
#transform independent variable using TF-IDF vectorizer
print("\n")
X_tfidf = tfidf.fit_transform(punc_text)
print("After vectorized text data\n\n",X_tfidf)



After vectorized text data

   (0, 73223)	0.20488795530960577
  (0, 41065)	0.27700555210200056
  (0, 39195)	0.2467207510781826
  (0, 2190)	0.2627509531217028
  (0, 51372)	0.27347410718435117
  (0, 38859)	0.2329718377146541
  (0, 39216)	0.2810823060707014
  (0, 72705)	0.2627509531217028
  (0, 71212)	0.199536006550559
  (0, 50090)	0.26757272965277223
  (0, 69552)	0.07866121219166562
  (0, 36594)	0.17599873978597042
  (0, 47725)	0.14314129205315726
  (0, 52506)	0.09076797056900891
  (0, 41053)	0.16411320984462158
  (0, 39175)	0.15262545043308065
  (0, 2145)	0.11648166017543675
  (0, 51371)	0.15135470160618467
  (0, 38637)	0.09027381162106451
  (0, 39212)	0.18150922378530784
  (0, 6427)	0.16885049252748643
  (0, 72629)	0.11566348098728918
  (0, 71103)	0.11487218350458195
  (0, 49981)	0.11680680878799281
  (0, 6445)	0.31692641392051796
  :	:
  (91131, 2447)	0.11703602441361614
  (91131, 14586)	0.12435794922281833
  (91131, 16032)	0.15400930299058735
  (91131, 21013)	0.12100103252576491
  

In [0]:
#Split the data into train and testing
X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state = 42)

In [20]:
#Print training data
print("\n")
print("Training data\n\n",X_train,"\n",Y_train)
print("\n\n")



Training data

   (0, 8745)	0.2621132479319096
  (0, 70538)	0.25501849426743817
  (0, 36344)	0.25501849426743817
  (0, 5948)	0.25501849426743817
  (0, 56543)	0.2583116168769591
  (0, 19771)	0.24501900060052614
  (0, 62891)	0.2153268811277455
  (0, 16307)	0.16498674249919792
  (0, 20539)	0.21354332445914204
  (0, 71617)	0.2216129792036079
  (0, 62)	0.20838674731679482
  (0, 55846)	0.19560344734054047
  (0, 62125)	0.1686951341771022
  (0, 15208)	0.12006252678525249
  (0, 56491)	0.12989420864733797
  (0, 8743)	0.1438744813203322
  (0, 61904)	0.17229594782674115
  (0, 76454)	0.1408980335726429
  (0, 36291)	0.13448338464617862
  (0, 65662)	0.16211761519009776
  (0, 10161)	0.13377314943911786
  (0, 49604)	0.15050867356068517
  (0, 5923)	0.13114091813041845
  (0, 71571)	0.12306135251513753
  (0, 56620)	0.1293786124634338
  :	:
  (63790, 31334)	0.20057161064884388
  (63790, 57543)	0.16910171196464405
  (63790, 2284)	0.19215790781978714
  (63790, 72035)	0.20499687570897399
  (63790, 13685)	0.

In [21]:
#Print testing data
print("Testing data\n\n",X_test)
print("\n\n")

Testing data

   (0, 2607)	0.22708671344507142
  (0, 51740)	0.2105665841889673
  (0, 30432)	0.19837620902792485
  (0, 49481)	0.216172311355276
  (0, 19005)	0.21413589505735037
  (0, 21745)	0.18014037519792403
  (0, 35782)	0.17936333686415132
  (0, 20076)	0.18356628821724735
  (0, 52752)	0.16572983422670431
  (0, 20413)	0.1850072520644142
  (0, 58060)	0.15859836994608553
  (0, 26558)	0.13345945898899236
  (0, 11421)	0.13845092046100088
  (0, 62480)	0.20136239054459854
  (0, 62474)	0.1406396911299062
  (0, 20406)	0.14731187460644507
  (0, 56057)	0.21887183128887208
  (0, 9949)	0.1454385895083515
  (0, 2598)	0.12781951852584628
  (0, 61957)	0.10377295210510014
  (0, 78254)	0.077506435883942
  (0, 48365)	0.1610428843124217
  (0, 77398)	0.15032360868663014
  (0, 42052)	0.09739457571746418
  (0, 4262)	0.11248617985437552
  :	:
  (27339, 56365)	0.07041064629777133
  (27339, 22047)	0.06636904952144532
  (27339, 51502)	0.07051225294299394
  (27339, 43390)	0.06269736719699623
  (27339, 21050)	0.

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
#Build the Naive_bayes model
clf = MultinomialNB()

In [24]:
#Fit train and test into the model
clf.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
#Predict the result
y_pred = clf.predict(X_test)

In [0]:
# print(y_pred[:100])

In [27]:
#classification report & confusion matrix
print("Confusion Matrix\n",confusion_matrix(Y_test,y_pred))
print("\n")
print("Classification Report\n",classification_report(Y_test,y_pred))
print("\n")
print("Accuracy : ",accuracy_score(Y_test,y_pred)*100)

Confusion Matrix
 [[ 8007  5732]
 [   30 13571]]


Classification Report
               precision    recall  f1-score   support

           0       1.00      0.58      0.74     13739
           1       0.70      1.00      0.82     13601

    accuracy                           0.79     27340
   macro avg       0.85      0.79      0.78     27340
weighted avg       0.85      0.79      0.78     27340



Accuracy :  78.92465252377468


In [0]:
import pickle

In [0]:

with open('/content/drive/My Drive/Colab Notebooks/training_models/mnb_trained_model.pkl', 'wb') as f:
    pickle.dump(clf, f)